In [2]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup

In [41]:
BASE_URL = "https://blokkenschema.nl"

In [14]:
res = r.get(BASE_URL+ '/lowlands-2023/line-up')

In [16]:
soup = BeautifulSoup(res.text, 'html.parser')
hl_links = [li.a['href'] for li in soup.find_all('li', class_='headliner')]

In [92]:
def get_schema_data(link):
    res_hl  = r.get(BASE_URL + link)
    soup_hl = BeautifulSoup(res_hl.text, 'html.parser')
    hl_data = soup_hl.find('div', class_='performance-data')
    return {
        'name': soup_hl.find('div', itemprop='performer').find('h3').string,
        'location': hl_data.find('div', itemprop='location').string,
        'start_date': hl_data.find('div', itemprop='startDate')['content'],
        'end_date': hl_data.find('span', itemprop='endDate')['content'],
    }

In [94]:
list_of_schema = [get_schema_data(link) for link in hl_links]
df = pd.DataFrame(list_of_schema)
df.to_csv("lowlands_schema.csv")

In [8]:
## write dataframe to a database with sqlalchemy engine
from sqlalchemy import create_engine
from config import Config
import pandas as pd

In [29]:
cfg = Config()
engine = create_engine(cfg.SQLALCHEMY_DATABASE_URI, echo=True)
# df.to_sql('artist', con=engine, if_exists='append', index=False)

In [30]:
# read rows from postgresql sqlalchemy query into pandas dataframe
engine = create_engine(cfg.SQLALCHEMY_DATABASE_URI, echo=False)
df = pd.read_sql_table('artist', engine)

In [34]:
df['starttime'] = df['startdate'].dt.strftime('%H:%M')
df['endtime'] = df['enddate'].dt.strftime('%H:%M')

In [35]:
df[df['stage'] == 'Alpha'].head(10)

,artist_id,name,stage,startdate,enddate,festival,day,starttime,endtime
31,31,Bicep,Alpha,2023-08-18 18:00:00,2023-08-18 19:15:00,Lowlands,Friday,18:00,19:15
32,32,Billie Eilish,Alpha,2023-08-20 21:45:00,2023-08-20 23:00:00,Lowlands,Sunday,21:45,23:00
38,38,Bombay Bicycle Club,Alpha,2023-08-20 15:40:00,2023-08-20 16:40:00,Lowlands,Sunday,15:40,16:40
46,46,Charlotte de Witte,Alpha,2023-08-19 22:45:00,2023-08-20 00:00:00,Lowlands,Saturday,22:45,00:00
64,64,De Jeugd Van Tegenwoordig,Alpha,2023-08-20 19:45:00,2023-08-20 20:45:00,Lowlands,Sunday,19:45,20:45
92,91,First Aid Kit,Alpha,2023-08-18 16:00:00,2023-08-18 17:00:00,Lowlands,Friday,16:00,17:00
94,93,Florence + the Machine,Alpha,2023-08-19 20:15:00,2023-08-19 21:30:00,Lowlands,Saturday,20:15,21:30
95,94,Foals,Alpha,2023-08-19 18:00:00,2023-08-19 19:00:00,Lowlands,Saturday,18:00,19:00
116,115,Joost,Alpha,2023-08-18 14:00:00,2023-08-18 15:00:00,Lowlands,Friday,14:00,15:00
154,153,Noord Nederlands Orkest,Alpha,2023-08-19 14:00:00,2023-08-19 15:00:00,Lowlands,Saturday,14:00,15:00


In [38]:
engine = create_engine(cfg.SQLALCHEMY_DATABASE_URI, echo=False)
df.to_sql('artist', con=engine, if_exists='append', index=False)

238